In [13]:
import pandas as pd
import requests
from itertools import combinations

In [14]:
api_url = 'https://fantasy.premierleague.com/api/bootstrap-static/'

response = requests.get(api_url)

if response.status_code == 200:
    data = response.json()
    print("Succesfully fetched data!")
else:
    print(f"Error while fetching data. Status code: {response.status_code}")

Succesfully fetched data!


In [15]:
print(data.keys())

dict_keys(['chips', 'events', 'game_settings', 'game_config', 'phases', 'teams', 'total_players', 'elements', 'element_stats', 'element_types'])


In [16]:
players = pd.DataFrame(data['elements'])
players.head()

,can_transact,can_select,chance_of_playing_next_round,chance_of_playing_this_round,code,cost_change_event,cost_change_event_fall,cost_change_start,cost_change_start_fall,dreamteam_count,...,now_cost_rank,now_cost_rank_type,form_rank,form_rank_type,points_per_game_rank,points_per_game_rank_type,selected_rank,selected_rank_type,starts_per_90,clean_sheets_per_90
0,True,False,0.0,0.0,438098,0,0,-1,1,0,...,137,82,674,304,682,310,636,278,0.00,0.00
1,True,True,100.0,100.0,205651,0,0,-2,2,0,...,32,15,303,38,459,49,213,37,0.74,0.37
2,True,True,75.0,75.0,226597,0,0,2,-2,2,...,66,3,76,17,31,4,13,5,1.08,0.33
3,True,True,100.0,100.0,219847,0,0,-1,1,1,...,13,5,88,18,37,14,37,10,1.01,0.29
4,True,False,0.0,0.0,463748,0,0,0,0,0,...,638,63,425,51,514,58,558,68,0.00,0.00


In [17]:
print("Players DataFrame Columns:")
print(players.columns.tolist())


Players DataFrame Columns:
['can_transact', 'can_select', 'chance_of_playing_next_round', 'chance_of_playing_this_round', 'code', 'cost_change_event', 'cost_change_event_fall', 'cost_change_start', 'cost_change_start_fall', 'dreamteam_count', 'element_type', 'ep_next', 'ep_this', 'event_points', 'first_name', 'form', 'id', 'in_dreamteam', 'news', 'news_added', 'now_cost', 'photo', 'points_per_game', 'removed', 'second_name', 'selected_by_percent', 'special', 'squad_number', 'status', 'team', 'team_code', 'total_points', 'transfers_in', 'transfers_in_event', 'transfers_out', 'transfers_out_event', 'value_form', 'value_season', 'web_name', 'region', 'minutes', 'goals_scored', 'assists', 'clean_sheets', 'goals_conceded', 'own_goals', 'penalties_saved', 'penalties_missed', 'yellow_cards', 'red_cards', 'saves', 'bonus', 'bps', 'influence', 'creativity', 'threat', 'ict_index', 'starts', 'expected_goals', 'expected_assists', 'expected_goal_involvements', 'expected_goals_conceded', 'influence_

In [30]:
position_map = {
    1: 'GK',
    2: 'Defender',
    3: 'Midfielder',
    4: 'Forward'
}

price_updates = {
    'Mohamed Salah': 13.0,
    'Cole Palmer': 10.8,
    'Bukayo Saka': 10.4,
    'Nicolas Jackson': 7.9, 
}

players_selected = players[
    ['first_name', 'second_name', 'team_code', 'element_type', 'total_points', 'points_per_game', 'now_cost']
].copy()

players_selected['now_cost'] = players_selected['now_cost'] / 10

players_selected['element_type'] = players_selected['element_type'].map(position_map)

players_selected[['total_points', 'points_per_game', 'now_cost']] = players_selected[
    ['total_points', 'points_per_game', 'now_cost']
].apply(pd.to_numeric, errors='coerce')

players_selected = players_selected.loc[players_selected['points_per_game'] > 0.0]

players_selected['full_name'] = players_selected['first_name'] + ' ' + players_selected['second_name']

# Remove Bryan Mbeumo's row from the DataFrame
#mbeumo_row = players_selected.loc[players_selected['full_name'] == 'Bryan Mbeumo'].copy()
#players_selected = players_selected.loc[players_selected['full_name'] != 'Bryan Mbeumo']

# Update player prices that have been already selected at a different price
for player, new_price in price_updates.items():
    players_selected.loc[players_selected['full_name'] == player, 'now_cost'] = new_price

players_selected['Cost/Points'] = (players_selected['now_cost'] / players_selected['total_points']).round(2)

print(players_selected)



         first_name             second_name  team_code element_type  \
1           Gabriel       Fernando de Jesus          3      Forward   
2           Gabriel    dos Santos Magalhães          3     Defender   
3               Kai                 Havertz          3      Forward   
5           Jurriën                  Timber          3     Defender   
6        Jorge Luiz            Frello Filho          3   Midfielder   
..              ...                     ...        ...          ...   
686          Jørgen           Strand Larsen         39      Forward   
687    Toti António                   Gomes         39     Defender   
689           André  Trindade da Costa Neto         39   Midfielder   
690  Carlos Roberto            Forbs Borges         39   Midfielder   
691           Alfie                    Pond         39     Defender   

     total_points  points_per_game  now_cost                     full_name  \
1               9              0.8       6.8     Gabriel Fernando de 

In [31]:
max_indeces = players_selected[['total_points', 'points_per_game', 'now_cost']].idxmax()
max_rows = players_selected.loc[max_indeces]
print(max_rows)


    first_name second_name  team_code element_type  total_points  \
401    Mohamed       Salah         14   Midfielder           151   
401    Mohamed       Salah         14   Midfielder           151   
428     Erling     Haaland         43      Forward            98   

     points_per_game  now_cost       full_name  Cost/Points  
401             10.8      13.0   Mohamed Salah         0.09  
401             10.8      13.0   Mohamed Salah         0.09  
428              6.5      14.9  Erling Haaland         0.15  


In [32]:
midfielders = players_selected[players_selected['element_type'] == 'Midfielder']

midfielders.tail()

,first_name,second_name,team_code,element_type,total_points,points_per_game,now_cost,full_name,Cost/Points
682,Daniel,Castelo Podence,39,Midfielder,2,1.0,5.5,Daniel Castelo Podence,2.75
683,Rodrigo,Martins Gomes,39,Midfielder,11,1.2,5.2,Rodrigo Martins Gomes,0.47
685,Pablo,Sarabia,39,Midfielder,10,2.2,5.1,Pablo Sarabia,0.51
689,André,Trindade da Costa Neto,39,Midfielder,16,1.5,5.0,André Trindade da Costa Neto,0.31
690,Carlos Roberto,Forbs Borges,39,Midfielder,5,0.8,5.5,Carlos Roberto Forbs Borges,1.10


In [33]:
print(f"Total midfielders: {len(midfielders)}")

Total midfielders: 229


In [34]:
min_indeces = midfielders[['total_points', 'points_per_game', 'now_cost']].idxmin()
min_rows = midfielders.loc[min_indeces]
print(min_rows)


    first_name   second_name  team_code element_type  total_points  \
24       Myles  Lewis-Skelly          3   Midfielder             1   
24       Myles  Lewis-Skelly          3   Midfielder             1   
356      Hamza     Choudhury         13   Midfielder             1   

     points_per_game  now_cost           full_name  Cost/Points  
24               0.5       4.5  Myles Lewis-Skelly          4.5  
24               0.5       4.5  Myles Lewis-Skelly          4.5  
356              1.0       4.3     Hamza Choudhury          4.3  


In [23]:
# Generate all combinations of 4 midfielders
defender_combinations = list(combinations(midfielders.itertuples(), 4))

print(len(defender_combinations))

109658025


In [26]:
print(mbeumo_row['now_cost'])

116    7.6
Name: now_cost, dtype: float64


In [28]:
optimal_midfielders = None
best_total_points = 0
best_comb_cost = 0
best_cpp = 100
max_cost = 50

for player1, player2, player3, player4 in defender_combinations:
    total_cost = player1.now_cost + player2.now_cost + player3.now_cost + player4.now_cost + 7.6
    if total_cost > max_cost:
        continue

    total_points = player1.total_points + player2.total_points + player3.total_points + player4.total_points + 96
    cpp = total_cost / total_points

    if total_points > best_total_points:
        best_cpp = cpp
        best_total_points = total_points
        best_comb_cost = total_cost
        optimal_midfielders = (player1, player2, player3, player4)

# Unpack and print the results
print(f"Total points: {best_total_points}")
print(f"Total cost: {best_comb_cost} \n")
print(f"Cost/point: {best_cpp} \n")

for player in optimal_midfielders:
    print(f"Player: {player.first_name} {player.second_name} - {player.element_type} - Team Code: {player.team_code} - Total points: {player.total_points} - Cost: £{player.now_cost}m - Cost / points: {player.now_cost / player.total_points}m")

print("Player: Bryan Mbeumo - Midfielder - Team Code: 94 - Total points: 96 - Cost: £7.6m - Cost / points: 0.07916666666666666m")



Total points: 551
Total cost: 47.50000000000001 

Cost/point: 0.08620689655172416 

Player: Bukayo Saka - Midfielder - Team Code: 3 - Total points: 103 - Cost: £10.4m - Cost / points: 0.10097087378640777m
Player: Cole Palmer - Midfielder - Team Code: 8 - Total points: 125 - Cost: £10.8m - Cost / points: 0.0864m
Player: Alex Iwobi - Midfielder - Team Code: 54 - Total points: 76 - Cost: £5.7m - Cost / points: 0.075m
Player: Mohamed Salah - Midfielder - Team Code: 14 - Total points: 151 - Cost: £13.0m - Cost / points: 0.08609271523178808m
Player: Bryan Mbeumo - Midfielder - Team Code: 94 - Total points: 96 - Cost: £7.6m - Cost / points: 0.07916666666666666m


In [39]:
# Create a sorted copy of the DataFrame
sorted_players = midfielders.sort_values(by='Cost/Points', ascending=True)
sorted_players = sorted_players.reset_index(drop=True)

# Display the sorted DataFrame
print(sorted_players)


    first_name             second_name  team_code element_type  total_points  \
0       Dwight                  McNeil         11   Midfielder            65   
1        Bryan                  Mbeumo         94   Midfielder            96   
2         Alex                   Iwobi         54   Midfielder            76   
3       Justin                Kluivert         91   Midfielder            62   
4       Mikkel               Damsgaard         94   Midfielder            58   
..         ...                     ...        ...          ...           ...   
224     Danilo  dos Santos de Oliveira         17   Midfielder             1   
225     Jeremy       Sarmiento Morante         36   Midfielder             1   
226     Samuel                  Edozie         20   Midfielder             1   
227     Carlos           Alcaraz Durán         20   Midfielder             1   
228   Federico                  Chiesa         14   Midfielder             1   

     points_per_game  now_cost         

In [44]:
# Knapsack algorithm

optimal_midfielders = list(sorted_players.head(5).index)
best_total_points = 0
best_comb_cost = 0
best_cpp = 100
max_cost = 48

for idx, row in sorted_players.iterrows():
    if idx not in optimal_midfielders:
        min_points = 10000
        min_points_player = None
        # Find player with minimum total points
        min_points_player = sorted_players.loc[optimal_midfielders, 'total_points'].idxmin()

        if row['total_points'] > sorted_players.loc[min_points_player, 'total_points']:
            worst_player_price = sorted_players.loc[min_points_player, 'now_cost']
            curr_cost = sorted_players.loc[optimal_midfielders, 'now_cost'].sum() - worst_player_price
            if curr_cost + row['now_cost'] <= max_cost:
                # Replace the optimal player
                optimal_midfielders.remove(min_points_player)
                optimal_midfielders.append(idx)

# Calculate team statistics
best_total_points = sorted_players.loc[optimal_midfielders, 'total_points'].sum()
best_comb_cost = sorted_players.loc[optimal_midfielders, 'now_cost'].sum()
best_cpp = best_comb_cost / best_total_points

# Print the team
print(f"Total points: {best_total_points}")
print(f"Total cost: £{best_comb_cost}m")
print(f"Cost/point: {best_cpp:.2f}m\n")

for player_idx in optimal_midfielders:
    player = sorted_players.loc[player_idx]
    print(f"Player: {player['first_name']} {player['second_name']} - {player['element_type']} - Team Code: {player['team_code']} - "
          f"Total points: {player['total_points']} - Cost: £{player['now_cost']}m - Cost / points: {player['now_cost'] / player['total_points']:.4f}m")


Total points: 551
Total cost: £47.5m
Cost/point: 0.09m

Player: Bryan Mbeumo - Midfielder - Team Code: 94 - Total points: 96 - Cost: £7.6m - Cost / points: 0.0792m
Player: Alex Iwobi - Midfielder - Team Code: 54 - Total points: 76 - Cost: £5.7m - Cost / points: 0.0750m
Player: Cole Palmer - Midfielder - Team Code: 8 - Total points: 125 - Cost: £10.8m - Cost / points: 0.0864m
Player: Mohamed Salah - Midfielder - Team Code: 14 - Total points: 151 - Cost: £13.0m - Cost / points: 0.0861m
Player: Bukayo Saka - Midfielder - Team Code: 3 - Total points: 103 - Cost: £10.4m - Cost / points: 0.1010m
